In [1]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
# from ROI_extraction import preprocess_image
import cv2
import os
# Set the path to dataset
dataset_path = '../images/3regimes/'

ids = []
labels = {}
classes = {'excess':1,'normal':0,'insufficient':2}
for class_name in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_name)
    if os.path.isdir(class_path):
        for filename in os.listdir(class_path) :
            if filename.endswith((".jpg", ".jpeg", ".png")):
                img_path = os.path.join(class_path, filename) 
                ids.append(img_path)
                labels[img_path]=classes[class_name]

# Set the input image dimensions
img_width, img_height = 100, 100
n_channels = 3

params = {'dim': (img_height,img_width),
          'batch_size': 64,
          'n_classes': 3,
          'n_channels': n_channels,
          'shuffle': False}

# Set the number of classes
num_classes = 3

def correct_gamma(image):
    # Convert image to float and normalize to range 0-1
    image_normalized = image.astype(float) / 255.0

    # Calculate mean R intensity
    meanRimg = np.mean(image_normalized[:, :, 2])  # Image is in BGR format
    
    # Calculate G value
    G = 0.74 * np.exp(-3.97 * meanRimg)
    
    # Apply transformation
    transformed_image = np.power(image_normalized, 1 / G)
    img_float32 = np.float32(transformed_image)
    return img_float32

def extract_ROI(original_image):
    # Convert to grayscale
    gray_image = cv2.cvtColor((original_image*255).astype(np.uint8), cv2.COLOR_BGR2GRAY)
    
    # # Apply histogram normalization
    # normalized_image = cv2.equalizeHist(gray_image)
    
    # Apply median filtering
    filtered_image = cv2.medianBlur(gray_image, 5)
    
    # Apply Otsu's thresholding
    _, thresholded_image = cv2.threshold(filtered_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Apply morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    closed_image = cv2.morphologyEx(thresholded_image, cv2.MORPH_CLOSE, kernel)
    opened_image = cv2.morphologyEx(closed_image, cv2.MORPH_OPEN, kernel)

    # Find contours in the processed image
    contours, _ = cv2.findContours(opened_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the contour with the largest area
    contour = max(contours, key=cv2.contourArea)
    
    # Calculate the moments of the contour
    M = cv2.moments(contour)
    
    # Calculate the center of the contour
    center_x = int(M["m10"] / M["m00"])
    center_y = int(M["m01"] / M["m00"])
    
    # Calculate the coordinates of the square ROI
    roi_size = img_height
    roi_x = center_x - roi_size // 2
    roi_y = center_y - roi_size // 2
    
    return {'contours':contours,'roi_x':roi_x,'roi_y':roi_y,'roi_size':roi_size}

def data_generation(list_IDs_temp):
    y = np.empty((len(list_IDs_temp)), dtype=int)
    X = []
    for i, ID in enumerate(list_IDs_temp):
        image = cv2.imread(ID)
        img_gamma_correct = correct_gamma(image)
        ROI = extract_ROI(img_gamma_correct)
        ROI = image[ROI['roi_y']:ROI['roi_y']+ROI['roi_size'], ROI['roi_x']:ROI['roi_x']+ROI['roi_size']]
        img = cv2.cvtColor(ROI, cv2.COLOR_BGR2HSV)
        H_val,S_val,V_val = img[:,:,0],img[:,:,1],img[:,:,2]
        # Extract the mean values of hue, saturation, and value
        H_mean = np.mean(H_val)
        S_mean = np.mean(S_val)
        V_mean = np.mean(V_val)
        # Extract the standard deviation of hue, saturation, and value
        H_std = np.std(H_val)
        S_std = np.std(S_val)
        V_std = np.std(V_val)
        # Extract the contrast values of hue, saturation, and value
        # H_max,S_max,V_max = np.max(H_val),np.max(S_val),np.max(V_val)
        # H_min,S_min,V_min = np.min(H_val),np.min(S_val),np.min(V_val)
        # H_contrast = (H_max - H_min) / (H_max + H_min)
        # S_contrast = (S_max - S_min) / (S_max + S_min)
        # V_contrast = (V_max - V_min) / (V_max + V_min)
        X.append([H_mean,H_std,S_mean,S_std,V_mean,V_std])
        y[i] = labels[ID]
    return pd.DataFrame(X,columns=['H_mean','H_std','S_mean','S_std','V_mean','V_std']),keras.utils.to_categorical(y, num_classes=params['n_classes']),y


In [2]:
X,y_categorical,y = data_generation(ids)

In [3]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
# Nombre de plis pour la validation croisée k-fold
k = 5

# Créer une instance de StratifiedKFold avec k plis
skf = StratifiedKFold(n_splits=k)


In [6]:
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
results = {i:{} for i in range(k)}

# Initialize a list to store the accuracy scores
accuracy_scores = []

for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")

    # Diviser les données d'entraînement et de validation pour ce pli
    X_train, X_val = X.iloc[train_index], X.iloc[test_index]

    y_train, y_val = y[train_index], y[test_index]

    # scaler = StandardScaler()
    # X_train = scaler.fit_transform(X_train)

    # Effectuer l'analyse en composantes principales (PCA)
    pca = PCA(n_components=2)  # Réduire le nombre de dimensions à 5
    X_train_pca = pca.fit_transform(X_train)
    X_val_pca = pca.transform(X_val)


    # Train the SVM classifier on the training data
    # Create a new instance of the SVM classifier for each fold
    svm_classifier = SVC()
    svm_classifier.fit(X_train_pca, y_train)

    y_val_pred = svm_classifier.predict(X_val_pca)

    # Calculate the accuracy score
    accuracy = metrics.accuracy_score(y_val, y_val_pred)

    # Obtenir les composantes principales
    components = pca.components_
    # Append the accuracy score to the list
    accuracy_scores.append(accuracy)
    # Afficher les composantes principales et leurs poids correspondants
    for i, component in enumerate(components):
        print("Composante", i+1)
        for j, weight in enumerate(component):
            print("Poids", j+1, ":", weight)
        print()
    print(f'Fold {i}: {accuracy}')
    
# Compute the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / k

# Print the average accuracy
print("Average Accuracy:", average_accuracy)

Fold 0:
  Train: index=[ 334  335  336 ... 5007 5008 5009]
  Test:  index=[   0    1    2 ... 3671 3672 3673]
Composante 1
Poids 1 : -0.09951514638472281
Poids 2 : -0.01871150538495736
Poids 3 : 0.21163357372313582
Poids 4 : -0.027681388361081843
Poids 5 : -0.9663372987455834
Poids 6 : 0.10190098856694878

Composante 2
Poids 1 : 0.23127618664095168
Poids 2 : 0.03990773468820333
Poids 3 : -0.709267419902875
Poids 4 : 0.09576090941250824
Poids 5 : -0.2469572755078834
Poids 6 : -0.6096723526382085

Fold 1: 0.8303393213572854
Fold 1:
  Train: index=[   0    1    2 ... 5007 5008 5009]
  Test:  index=[ 334  335  336 ... 4005 4006 4007]
Composante 1
Poids 1 : -0.10432396372413619
Poids 2 : -0.019531050614095102
Poids 3 : 0.22777147940950565
Poids 4 : -0.03001387876198143
Poids 5 : -0.9611586744815345
Poids 6 : 0.11012888527975263

Composante 2
Poids 1 : 0.23463550218816986
Poids 2 : 0.04052849002148487
Poids 3 : -0.7145332064639196
Poids 4 : 0.0987728786481448
Poids 5 : -0.26666992174380677
P

In [5]:
# # Tracer les données d'entraînement et de validation ainsi que les frontières de décision
# x_min, x_max = X_train_pca[:, 0].min() - 1, X_train_pca[:, 0].max() + 1
# y_min, y_max = X_train_pca[:, 1].min() - 1, X_train_pca[:, 1].max() + 1
# xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1), np.arange(y_min, y_max, 0.1))
# Z = svm_classifier.predict(np.c_[xx.ravel(), yy.ravel()])
# Z = Z.reshape(xx.shape)


# plt.figure()
# plt.contourf(xx, yy, Z, alpha=0.8)

# # Tracer les échantillons de validation en utilisant des marqueurs distincts
# plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1], c=y_train, edgecolor='k', label='Entraînement')
# plt.scatter(X_val_pca[:, 0], X_val_pca[:, 1], c='gray', marker='x', edgecolor='k', label='Validation')

# plt.xlabel('Composante principale 1')
# plt.ylabel('Composante principale 2')
# plt.title('Classification avec SVM')
# plt.legend()
# plt.show()